# **Importing Libraries**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
import re
import time
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import string
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import plotly.figure_factory as ff
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
import scipy.io
from sklearn.feature_selection import mutual_info_regression

# **Normalization function**

In [ ]:
def normalize(df):
  for column in df.columns:
    df[column] = (df[column] - df[column].mean()) / df[column].std()    
  return df

# **Data Handling**

In [ ]:
def read(url):
  mat = scipy.io.loadmat(url)
  l=list(mat.values())
  df = pd.DataFrame(l[-1], columns =['C1', 'C2', 'C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19'])
  return df

**Reading the files inside ADHD and Control ADHD Folders**

In [ ]:
import os

def get_filepaths(directory):

    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  # Add it to the list.

    return file_paths
file_paths1 = get_filepaths("/content/drive/MyDrive/Colab Notebooks/ADHD_part1")
file_paths2 = get_filepaths("/content/drive/MyDrive/Colab Notebooks/ADHD_part2")
file_paths3 = get_filepaths("/content/drive/MyDrive/Colab Notebooks/Control_part1")
file_paths4 = get_filepaths("/content/drive/MyDrive/Colab Notebooks/Control_part2")

**Finding the minimum number of rows**

In [ ]:
AP1=[]
for i in file_paths1:
  df=read(i)
  r,c=df.shape
  AP1.append(r)

In [ ]:
print(len(AP1))
for i in file_paths2:
  df=read(i)
  r,c=df.shape
  AP1.append(r)

30


In [ ]:
print(len(AP1))
for i in file_paths3:
  df=read(i)
  r,c=df.shape
  AP1.append(r)

61


In [ ]:
print(len(AP1))
for i in file_paths4:
  df=read(i)
  r,c=df.shape
  AP1.append(r)

91


In [ ]:
len(AP1)

121

In [ ]:
AP1.sort()

In [ ]:
print("Minimum number of rows is",AP1[0])

Minimum number of rows is 7983


**Reducing the dimension of each subject's matrix to min_rows x 19**

In [ ]:
def split(url):
  mat = scipy.io.loadmat(url)
  l=list(mat.values())
  df = pd.DataFrame(l[-1], columns =['C1', 'C2', 'C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19'])
  return df[0:AP1[0]]

In [ ]:
AP1_df=[]
for i in file_paths1:
  df=split(i)
  #r,c=df.shape
  df=normalize(df)
  AP1_df.append(df)

In [ ]:
AP2_df=[]
for i in file_paths2:
  df=split(i)
  #r,c=df.shape
  df=normalize(df)
  AP2_df.append(df)

In [ ]:
AP3_df=[]
for i in file_paths3:
  df=split(i)
  #r,c=df.shape
  df=normalize(df)
  AP3_df.append(df)

In [ ]:
AP4_df=[]
for i in file_paths4:
  df=split(i)
  #r,c=df.shape
  df=normalize(df)
  AP4_df.append(df)

In [ ]:
AP1_MI=[]
AP1_MMI=[]
for i in AP1_df:
  arr=i.to_numpy()
  mi = np.array([mutual_info_regression(arr,arr[:, i]) for i in range(19)])
  d_vec = np.mean(mi, axis=0)
  AP1_MI.append(mi)
  AP1_MMI.append(d_vec)

**Finding average mutual information after finding mutual information between every column to obtain a 19 x 1 matrix for each subject**

In [ ]:
AP2_MI=[]
AP2_MMI=[]
for i in AP2_df:
  arr=i.to_numpy()
  mi = np.array([mutual_info_regression(arr,arr[:, i]) for i in range(19)])
  d_vec = np.mean(mi, axis=0)
  AP2_MI.append(mi)
  AP2_MMI.append(d_vec)

In [ ]:
AP3_MI=[]
AP3_MMI=[]
for i in AP3_df:
  arr=i.to_numpy()
  mi = np.array([mutual_info_regression(arr,arr[:, i]) for i in range(19)])
  d_vec = np.mean(mi, axis=0)
  AP3_MI.append(mi)
  AP3_MMI.append(d_vec)

In [ ]:
AP4_MI=[]
AP4_MMI=[]
for i in AP4_df:
  arr=i.to_numpy()
  mi = np.array([mutual_info_regression(arr,arr[:, i]) for i in range(19)])
  d_vec = np.mean(mi, axis=0)
  AP4_MI.append(mi)
  AP4_MMI.append(d_vec)

In [ ]:
AP1_MMI[0].shape

(19,)

In [ ]:
AP1_MM=[l.tolist() for l in AP1_MMI]

In [ ]:
AP2_MM=[l.tolist() for l in AP2_MMI]
AP3_MM=[l.tolist() for l in AP3_MMI]
AP4_MM=[l.tolist() for l in AP4_MMI]

In [ ]:
df_prepro=pd.DataFrame(data=AP1_MM,columns =['C1', 'C2', 'C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19'])

In [ ]:
df_prepro2=pd.DataFrame(data=AP2_MM,columns =['C1', 'C2', 'C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19'])

In [ ]:
df_prepro3=pd.DataFrame(data=AP3_MM,columns =['C1', 'C2', 'C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19'])

In [ ]:
df_prepro4=pd.DataFrame(data=AP4_MM,columns =['C1', 'C2', 'C3','C4','C5','C6','C7','C8','C9','C10','C11','C12','C13','C14','C15','C16','C17','C18','C19'])

**Getting the final preprocessed csv file**

In [ ]:
df_finalcsv=pd.concat([df_prepro, df_prepro2, df_prepro3,df_prepro4], ignore_index=True)

In [ ]:
label=[]
for i in range(0,121):
  if(i<61):
    label.append(0)
  else:
    label.append(1)

In [ ]:
df_finalcsv['label']=label

In [ ]:
df_finalcsv

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,label
0,0.335038,0.350803,0.386383,0.382694,0.388903,0.384259,0.324582,0.371772,0.178842,0.235855,0.287295,0.297958,0.312892,0.258505,0.310902,0.313029,0.306131,0.313667,0.310917,0
1,0.387232,0.274562,0.494701,0.290063,0.506105,0.245683,0.474407,0.287532,0.462121,0.228246,0.428885,0.250042,0.417819,0.243313,0.394529,0.211636,0.484410,0.253041,0.430822,0
2,0.213529,0.226200,0.233447,0.239186,0.206145,0.269256,0.207142,0.280646,0.200542,0.270002,0.184769,0.167180,0.183914,0.194079,0.160464,0.183387,0.229746,0.257888,0.227645,0
3,0.292671,0.241536,0.493374,0.502810,0.516281,0.521020,0.417825,0.456831,0.282943,0.267064,0.245093,0.258088,0.337609,0.403353,0.297118,0.377677,0.437617,0.342920,0.378390,0
4,0.346029,0.354078,0.360827,0.449844,0.383315,0.354668,0.371950,0.437448,0.215580,0.384313,0.329467,0.401644,0.274604,0.388668,0.307233,0.399812,0.309470,0.376372,0.309475,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,0.309895,0.312626,0.230568,0.349177,0.328292,0.318996,0.358948,0.341061,0.288601,0.306197,0.287438,0.318433,0.249575,0.299025,0.247730,0.228662,0.281399,0.241214,0.252185,1
117,0.318146,0.284052,0.255549,0.289795,0.207993,0.242576,0.213281,0.226987,0.175099,0.215026,0.283096,0.203045,0.249787,0.182652,0.159298,0.144923,0.278210,0.248315,0.199927,1
118,0.301197,0.270250,0.310468,0.298029,0.292355,0.261588,0.261356,0.251037,0.188390,0.209553,0.248784,0.162316,0.250101,0.221374,0.189878,0.209652,0.288664,0.244984,0.210675,1
119,0.413807,0.306365,0.372274,0.266940,0.384445,0.296911,0.406224,0.248007,0.261596,0.226670,0.382783,0.227374,0.411393,0.270862,0.400743,0.261594,0.388536,0.271187,0.364075,1


In [ ]:
df_finalcsv.to_csv('Df_preprocessed.csv')